In [ ]:
!pip install duckdb --pre --upgrade

In [2]:
sf =100

StatementMeta(, bd700cbb-3a21-4159-a7e0-8e12ae495dad, 4, Finished, Available)

In [3]:
%%time
import duckdb
import pathlib
for x in range(0, sf) :
  con=duckdb.connect()
  con.sql('PRAGMA disable_progress_bar;SET preserve_insertion_order=false')
  con.sql(f"CALL dbgen(sf={sf} , children ={sf}, step = {x})") 
  for tbl in ['nation','region','customer','supplier','lineitem','orders','partsupp','part'] :
     pathlib.Path(f'/lakehouse/default/Files/{sf}/{tbl}').mkdir(parents=True, exist_ok=True) 
     con.sql(f"COPY (SELECT * FROM {tbl}) TO '/lakehouse/default/Files/{sf}/{tbl}/{x:02d}.parquet' ")
  con.close()

StatementMeta(, bd700cbb-3a21-4159-a7e0-8e12ae495dad, 5, Submitted, Running)

CPU times: user 56min 36s, sys: 2min 31s, total: 59min 8s
Wall time: 25min 50s


In [4]:
spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")

StatementMeta(, , , Waiting, )

In [5]:
from pyspark.sql.types import *
def loadFullDataFromSource(table_name):
    df = spark.read.parquet(f'Files/{sf}/' + table_name + '/*.parquet')
    df.write.mode("overwrite").format("delta").save(f"Tables/" + table_name)
full_tables = [
    'customer',
    'lineitem',
    'nation',
    'orders' ,
    'region',
    'partsupp',
    'supplier' ,
    'part'
    ]

for table in full_tables:
    loadFullDataFromSource(table)

StatementMeta(, , , Waiting, )